In [8]:
from binance.client import Client
import pandas as pd
import ta
import time
import gate_api
import requests
from gate_api.exceptions import ApiException, GateApiException
import numpy as np

# Binance API client oluştur
client = Client(api_key='key', api_secret='secret')


# Sembol ve zaman dilimini ayarla
SYMBOL = 'BTC_USDT'
BINANCE_SYMBOL = 'BTCUSDT'

# Son 100 mumu al
klines = client.get_historical_klines(BINANCE_SYMBOL, '1m', '50001 minutes ago UTC')
closess = np.array([float(kline[4]) for kline in klines])

def calculate_ema(closes, period):
    ema = np.zeros_like(closes)
    ema[0] = closes[0]  # Başlangıç değeri, ilk kapanış fiyatıyla aynı
    # EMA hesaplaması
    alpha = 2 / (period + 1)
    for i in range(1, len(closes)):
        ema[i] = closes[i] * alpha + ema[i - 1] * (1 - alpha)
    return ema

opens = []
closes = []
highs = []
lows = []

# Her mum için verileri topla
for kline in klines:
    opens.append(float(kline[1]))
    closes.append(float(kline[4]))
    highs.append(float(kline[2]))
    lows.append(float(kline[3]))

# Verileri bir DataFrame'e dönüştür
data = {
    'Açılış': opens,
    'Kapanış': closes,
    'En Yüksek': highs,
    'En Düşük': lows
}
df = pd.DataFrame(data)

# EMA7, EMA25 ve EMA99 hesapla ve veri çerçevesine ekle
ema7 = calculate_ema(closes, 7)
ema25 = calculate_ema(closes, 25)
ema99 = calculate_ema(closes, 99)
df['EMA7'] = ema7
df['EMA25'] = ema25
df['EMA99'] = ema99



In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, f1_score
import matplotlib.pyplot as plt

# Veri setini yükleme
df = pd.read_csv('modelleme.csv')
df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')


# Özellikler ve hedef değişkeni ayrıştırma
X = df[['Açılış', 'Kapanış', 'En Yüksek', 'En Düşük', 'RSI', 'EMA7', 'EMA25', 'EMA99']]
y = df['Fiyat_Artis']

# Test ve eğitim setlerini oluşturma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost modeli oluşturma
model = XGBClassifier()

seed = 42

param_grid = {
    'eta': [0.01, 0.05],
    'max_depth': [9, 12, 15],
    'min_child_weight': [1, 5, 10, 50, 250, 500, 750],
    'subsample': [1],
    'colsample_bytree': [1],
    'gamma': [0, 40, 80],
    'reg_alpha': [0.9, 0.5, 0.1],
    'objective': ["binary:logistic"],
    'eval_metric': ["logloss"],
    'booster': ["gbtree"],
    'seed': [seed]
}

# K-Fold oluşturma
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# GridSearchCV ile parametre optimizasyonu yapma
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=kfold, verbose=2)
grid_result = grid_search.fit(X_train, y_train)

# En iyi parametrelerin çıktısı
print("En iyi parametreler:", grid_result.best_params_)

# Tahmin yapma
y_pred = grid_result.predict(X_test)

# Performans değerlendirme metrikleri
accuracy = accuracy_score(y_test, y_pred)
print("Test seti doğruluğu:", accuracy)

roc_auc = roc_auc_score(y_test, grid_result.predict_proba(X_test)[:, 1])
print("Modelin ROC AUC Skoru:", roc_auc)

gini_impurity_score = 2 * roc_auc - 1
print("Modelin Gini Skoru:", gini_impurity_score)

# Hassasiyet-recall eğrisi
precision, recall, _ = precision_recall_curve(y_test, grid_result.predict_proba(X_test)[:, 1])
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True)
plt.show()

# F1 skoru
f1 = f1_score(y_test, y_pred)
print("Modelin F1 Skoru:", f1)



KeyboardInterrupt



In [94]:
import pickle

# Eğitilmiş modeli dosyaya kaydetme
with open('xgboost_model.pkl', 'wb') as f:
    pickle.dump(grid_result.best_estimator_, f)

In [3]:
import pickle
# Kaydedilmiş modeli yükleme
with open('xgboost_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)


In [4]:
from binance.client import Client
import pandas as pd
import ta
import time
import gate_api
import requests
from gate_api.exceptions import ApiException, GateApiException
import numpy as np

# Binance API client oluştur
client = Client(api_key='key', api_secret='secret')


# Sembol ve zaman dilimini ayarla
SYMBOL = 'BTC_USDT'
BINANCE_SYMBOL = 'BTCUSDT'

# Son 100 mumu al
klines = client.get_historical_klines(BINANCE_SYMBOL, '1m', '100 minutes ago UTC')

def calculate_ema(closes, period):
    ema = np.zeros_like(closes)
    ema[0] = closes[0]  # Başlangıç değeri, ilk kapanış fiyatıyla aynı
    # EMA hesaplaması
    alpha = 2 / (period + 1)
    for i in range(1, len(closes)):
        ema[i] = closes[i] * alpha + ema[i - 1] * (1 - alpha)
    return ema

def calculate_rsi(closes, period=6):
    deltas = np.diff(closes)
    seed = deltas[:period]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down
    rsi = np.zeros_like(closes)
    rsi[:period] = 100. - 100. / (1. + rs)
    for i in range(period, len(closes)):
        delta = deltas[i - 1]  # `i` öğesiyle bir önceki kapanış arasındaki fark
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period
        rs = up / down
        rsi[i] = 100. - 100. / (1. + rs)
    return rsi[-1]

opens = []
closes = []
highs = []
lows = []

# Her mum için verileri topla
for kline in klines:
    opens.append(float(kline[1]))
    closes.append(float(kline[4]))
    highs.append(float(kline[2]))
    lows.append(float(kline[3]))

# Verileri bir DataFrame'e dönüştür
data = {
    'Açılış': opens,
    'Kapanış': closes,
    'En Yüksek': highs,
    'En Düşük': lows
}
df = pd.DataFrame(data)

rsi = calculate_rsi(closes, period=6)
df['RSI'] = rsi

# EMA7, EMA25 ve EMA99 hesapla ve veri çerçevesine ekle
ema7 = calculate_ema(closes, 7)
ema25 = calculate_ema(closes, 25)
ema99 = calculate_ema(closes, 99)

df['EMA7'] = ema7
df['EMA25'] = ema25
df['EMA99'] = ema99
test = df
#test = test.drop(columns=['Unnamed: 0'])



In [11]:
new_predictions = loaded_model.predict(tt)
class_probabilities = loaded_model.predict_proba(tt)
custom_predictions = np.max(class_probabilities, axis=1)

In [12]:
kk = pd.DataFrame(custom_predictions)
kk.rename(columns={0: 'CustomPredictions'}, inplace=True)
ss = pd.DataFrame(new_predictions)
ss.rename(columns={0: 'ModelPredictions'}, inplace=True)

In [17]:
merged_df = pd.concat([tt, ss], axis=1)
merged_df1 = pd.concat([merged_df, kk], axis=1)

In [18]:
merged_df1[0:50]

,Açılış,Kapanış,En Yüksek,En Düşük,RSI,EMA7,EMA25,EMA99,ModelPredictions,CustomPredictions
0,43043.99,43043.98,43043.99,43039.16,66.790516,43036.849722,43034.192952,42999.564613,0,0.584370
1,43043.98,43043.98,43043.98,43043.97,66.790516,43038.632292,43034.945802,43000.452920,0,0.584370
2,43043.98,43043.97,43043.98,43043.96,66.753451,43039.966719,43035.639971,43001.323262,0,0.584370
3,43043.97,43057.98,43057.99,43043.96,82.800199,43044.470039,43037.358434,43002.456397,0,0.530889
4,43057.99,43068.79,43068.79,43057.98,88.112644,43050.550029,43039.776247,43003.783069,0,0.543212
5,43068.78,43062.92,43068.79,43062.91,73.349980,43053.642522,43041.556536,43004.965807,0,0.559961
6,43062.91,43062.92,43062.92,43062.91,73.349980,43055.961892,43043.199879,43006.124891,0,0.559961
7,43062.92,43051.79,43062.92,43051.10,50.327526,43054.918919,43043.860658,43007.038193,0,0.560427
8,43051.79,43051.78,43051.79,43051.78,50.310500,43054.134189,43044.469838,43007.933030,0,0.560427
9,43051.78,43051.79,43051.79,43051.78,50.330663,43053.548142,43045.032927,43008.810169,0,0.560427
